## SNEAKSAVER IMAGE RECOGINATION TOOL USING DECISION TREE

In [62]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import os
from skimage import io, transform, exposure, util, color
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

In [63]:

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("")))

inputDir =  os.path.join(BASE_DIR, "media", "dataset", "sneakers")
outputDir = os.path.join(BASE_DIR, "media", "dataset", "preprocess")
target = []
flat_data = []
images = []
DataDirectory =  outputDir

# Images to be classified as:
Categories = ["nike","converse","adidas"]

In [64]:
def image_augmentation(image, target_shape=(240, 240, 3)):
    
    """Image Augmentation

    Returns:
        _type_: _description_
    """
    # Rotation
    angle = np.random.uniform(low=-40, high=40)
    image = transform.rotate(image, angle)

    # Shift
    width_shift = np.random.uniform(low=-0.2, high=0.2)
    height_shift = np.random.uniform(low=-0.2, high=0.2)
    image = transform.warp(image, transform.AffineTransform(translation=(width_shift, height_shift)))

    # Shear
    shear_range = np.random.uniform(low=-0.2, high=0.2)
    image = transform.warp(image, transform.AffineTransform(shear=shear_range))

    # Zoom
    zoom_range = np.random.uniform(low=0.8, high=1.2)
    image = transform.rescale(image, zoom_range)

    # Horizontal Flip
    if np.random.choice([True, False]):
        image = np.fliplr(image)

    # Fill mode 'nearest' - scikit-image warp doesn't have a direct equivalent
    # You may choose to use skimage.util.random_noise or other transformations for filling
    
    augmented_image = (image * 255).astype(np.uint8)
    
    # Convert to RGB if the image is in grayscale with an alpha channel
    if augmented_image.shape[-1] == 2:
        augmented_image = color.gray2rgb(augmented_image[:, :, 0])
        
    # Convert to RGB if the image is in RGBA format
    if augmented_image.shape[-1] == 4:
        augmented_image = color.rgba2rgb(augmented_image)
        
     # Ensure the image has 3 channels (RGB)
    if augmented_image.shape[-1] != 3:
        augmented_image = np.stack([augmented_image[:, :, 0]] * 3, axis=-1)
        
    # Resize the image to the target shape
    augmented_image = transform.resize(augmented_image, target_shape, anti_aliasing=True)

    augmented_image = util.img_as_ubyte(augmented_image)
    
        
    
        
    #print(f"Augmented image shape: {augmented_image.shape}")
    #print(f"Augmented image data type: {augmented_image.dtype}")

        
    return augmented_image



def dataset_augmentation(input_dir, output_dir):
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate through each image in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"): 
            # Add more file extensions as needed
            
            image_path = os.path.join(input_dir, filename)
            original_image = io.imread(image_path)

            # Apply augmentation
            augmented_image = image_augmentation(original_image)

            # Save the augmented image to the output directory
            output_path = os.path.join(output_dir, filename)
            io.imsave(output_path, augmented_image)

In [65]:


for i in Categories:
    dataset_augmentation(os.path.join(inputDir, i), os.path.join(outputDir, i))

In [66]:
for i in Categories:
  print("Category is:",i,"\tLabel encoded as:",Categories.index(i))
  # Encode categories cute puppy as 0, icecream cone as 1 and red rose as 2
  target_class = Categories.index(i)
  # Create data path for all folders under MinorProject
  path = os.path.join(DataDirectory,i)
  # Image resizing, to ensure all images are of same dimensions
  for img in os.listdir(path):
    img_array = imread(os.path.join(path,img))
    # Skimage normalizes the value of image
    img_resized = resize(img_array,(240,240,3))
    flat_data.append(img_resized.flatten())
    images.append(img_resized)
    target.append(target_class)
# Convert list to numpy array format
flat_data = np.array(flat_data)
images = np.array(images)
target = np.array(target)

Category is: nike 	Label encoded as: 0


Category is: converse 	Label encoded as: 1
Category is: adidas 	Label encoded as: 2


In [67]:

df = pd.DataFrame(flat_data)
# Create a column for output data called Target
df['Target'] = target
# Rows are all the input images (90 images, 30 of each category)
df

X = df.drop('Target', axis=1)
y = df['Target']

In [69]:
# Assuming X contains your features and y contains your labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a decision tree classifier
model = DecisionTreeClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Accuracy: 0.35080645161290325
              precision    recall  f1-score   support

           0       0.32      0.35      0.33        84
           1       0.41      0.33      0.36        86
           2       0.34      0.38      0.36        78

    accuracy                           0.35       248
   macro avg       0.36      0.35      0.35       248
weighted avg       0.36      0.35      0.35       248

